## PydanticOutputParser
**강력하고 유연한 OutputParser**로, Pydantic 모델을 사용하여 복잡한 구조화된 데이터를 파싱합니다.

**주요 특징:**

- **타입 검증 자동화**: 각 필드의 데이터 타입을 자동으로 검증
- **중첩된 객체 구조 지원**: 복잡한 계층 구조의 데이터 처리 가능
- **자동 스키마 생성**: Pydantic 모델에서 자동으로 JSON 스키마 생성
- **에러 처리 내장**: 잘못된 형식에 대한 상세한 에러 메시지 제공

**활용 사례:**

- API 응답 구조화
- 복잡한 설문조사 결과 파싱
- 제품 정보, 사용자 프로필 등 다중 필드 데이터 처리
- 데이터베이스 스키마와 연동되는 구조화된 정보 추출

In [5]:
from langchain.output_parsers import PydanticOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

from pydantic import BaseModel, Field
from typing import List

In [6]:
# 출력 구조를 정의하는 Pydantic 모델
class MovieRecommendation(BaseModel):
    movie_title: str = Field(description="추천 영화 제목")
    reason: str = Field(description="추천 이유")
    genre: List[str] = Field(description="영화 장르")
    estimated_rating: float = Field(description="10점 만점에서 예상 평점")

In [7]:
# Pydantic 출력 파서 초기화
parser = PydanticOutputParser(pydantic_object=MovieRecommendation)

In [8]:
# 프롬프트 템플릿 설정
template = """
다음 사용자 요청에 따라 영화를 추천해주세요.
요청: {query}

{format_instructions}
"""

In [9]:
prompt = ChatPromptTemplate.from_template(template)

# 파서의 지시사항을 프롬프트에 주입
prompt = prompt.partial(
    format_instructions=parser.get_format_instructions()
)

In [10]:
# ChatOpenAI 모델 초기화
#model = ChatOpenAI(temperature=0.7, model="gpt-3.5-turbo")
model = ChatOpenAI(
    #api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",  # Spring AI와 동일한 모델
    temperature=0.7
)

# 체인 구성 및 실행
query = "1990년대 클래식한 느낌의 공포 영화 추천해줘"
chain = prompt | model | parser
output = chain.invoke({"query": query})

# 결과 출력
print(f"추천 영화: {output.movie_title}")
print(f"추천 이유: {output.reason}")
print(f"장르: {', '.join(output.genre)}")
print(f"예상 평점: {output.estimated_rating}/10")

추천 영화: The Silence of the Lambs
추천 이유: 1990년대 클래식한 느낌의 공포 영화로, 긴장감과 스릴러 요소가 가득한 작품입니다.
장르: 공포, 스릴러, 미스터리
예상 평점: 9.5/10
